In [ ]:
from selenium.webdriver.common.by import By
import win32com.client as win32
from selenium import webdriver
import pandas as pd
import time


def formatar_preco(preco: str) -> int:
    preco = preco.replace(',','').replace('.','').replace(' ','').replace('R$','')
    if 'amazon' in link:
        preco = preco + '00'
    preco = preco[:-2]
    preco = int(preco)
    return preco


def menor_valor_produto(lista:list) -> tuple:
    precos_menor = ()
    for produto_loja, preco, endereco in lista:
        if precos_menor == ():
            precos_menor = (produto_loja, preco, endereco)
        elif preco < precos_menor[1]:
            precos_menor = (produto_loja, preco, endereco)
    return precos_menor


while True:
    # Importando planilha com o nome, endereços e vaolr atual dos produtos
    planilha = pd.read_excel('arquivos/produtos.xlsx')
    planilha_menor_preco = planilha[['Produto', 'Preço Original']]

    # Realiza o web scraping dos preços dos produtos, dentro dos endereços web contidos planilha
    drive = webdriver.Chrome()
    precos_memoria = []
    precos_tv = []
    for linha in range(len(planilha['Produto'])):
        for coluna in range(1, 4):
            link = planilha.iloc[linha][coluna]
            drive.get(f'{link}')
            time.sleep(5)
            try:
                if 'kabum' in link and 'memoria' in link:
                    valor_memoria_kabum = formatar_preco(drive.find_element(By.XPATH, '//*[@id="blocoValores"]/div[2]/div[1]/h4').text)
                    precos_memoria.append(('Kabum', valor_memoria_kabum, link))
                elif 'pichau' in link and 'memoria' in link:
                    valor_memoria_pichau = formatar_preco(drive.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div/div[2]/div[4]/div[1]/div/div[1]/div[2]/div[2]').text)
                    precos_memoria.append(('Pichau', valor_memoria_pichau, link))
                elif 'terabyteshop' in link and 'memoria' in link:
                    valor_memoria_terabyte = formatar_preco(drive.find_element(By.XPATH, '//*[@id="valVista"]').text)
                    precos_memoria.append(('Terabyte', valor_memoria_terabyte, link))
                elif 'casasbahia' in link and 'smart-tv' in link:
                    valor_tv_casasbahia = formatar_preco(drive.find_element(By.XPATH, '//*[@id="product-price"]').text)
                    precos_tv.append(('Casas Bahia', valor_tv_casasbahia, link))
                elif 'amazon' in link and 'Smart-LED' in link:
                    valor_tv_amazon = formatar_preco(drive.find_element(By.XPATH, '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span/span[2]/span[2]').text)
                    precos_tv.append(('Amazon', valor_tv_amazon, link))
                elif 'extra' in link and 'smart-tv' in link:
                    valor_tv_extra = formatar_preco(drive.find_element(By.XPATH, '//*[@id="product-price"]').text)
                    precos_tv.append(('Extra', valor_tv_extra, link))
            except:
                print('\tAlgum produto esta indisponivel, necessario verificar os links\n')

    # Determina qual loja esta com o menor preço junto a função
    preco_menor_memoria = menor_valor_produto(precos_memoria)
    preco_tv_menor = menor_valor_produto(precos_tv)

    # Adiciona na DataFrame planilha menor preço a loja, preço e link da melhor oferta disponivel
    link_mvalo_planilha = [preco_menor_memoria[2], preco_tv_menor[2]]
    menor_valor_planilha = [preco_menor_memoria[1], preco_tv_menor[1]]
    loja_mvalor_planilha = [preco_menor_memoria[0], preco_tv_menor[0]]

    planilha_menor_preco['Menor Preço'] = menor_valor_planilha
    planilha_menor_preco['Melhor Compra'] = loja_mvalor_planilha
    planilha_menor_preco['Link Produto'] = link_mvalo_planilha

    # Salva copia de planilha menor preço a loja, preço e link para consulta
    planilha_menor_preco.to_excel('arquivos/produtos_menor_preco.xlsx')
    print('\tPlanilha com menores valores salva.\n')

    # Envia e-mail caso alguns dos produtos estiver entrado empreomoção com preço abaixo de 20% do original
    outlook = win32.Dispatch('outlook.application')

    for produto in range(len(planilha_menor_preco['Produto'])):
        if  planilha_menor_preco['Preço Original'][produto] * 0.80 < planilha_menor_preco['Menor Preço'][produto]:

            corpo_email = f'''
            <body>
                <div style="font-family: Helvetica; text-align: center">
                        <h2 style="color: red"><p>Atenção!</p></h2>

                        <p>O produto {planilha_menor_preco['Produto'][produto]} entrou em promoção no(a)
                            {planilha_menor_preco['Melhor Compra'][produto]},<br>com mais de 20% de desconto.</p>

                        <p>A(o) {planilha_menor_preco['Produto'][produto]} saiu de
                            R$ {planilha_menor_preco['Preço Original'][produto]},00
                            para R$ {planilha_menor_preco['Menor Preço'][produto]},00.</p>

                        <span style="text-align: center"><p><b>Não perca esssa opotunidade!</b></p></span>

                        <p>E acesse o link do(a) {planilha_menor_preco['Produto'][produto]} do(a)
                            {planilha_menor_preco['Melhor Compra'][produto]} abaixo.<br>

                        {planilha_menor_preco['Link Produto'][produto]}

                </div>
            </body>
            '''

            mail = outlook.CreateItem(0)
            mail.To = 'romuloptmota@gmail.com'
            mail.Subject = f"{planilha_menor_preco['Produto'][produto]} entrou em Promoção!"
            mail.HTMLBody = corpo_email

            mail.Send()
        else:
            print(f"\tO produto {planilha_menor_preco['Produto'][produto]} não entrou em promoção\n")

    # temporizado para roda a cada trÊs horas para verificação dos preços
    print('\tLoop realizado!')
    time.sleep((60*60)*3)


	Algum produto esta indisponivel, necessario verificar os links



D:\Documentos\Projetos_para_Github\Automacao_Web\venv\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Documentos\Projetos_para_Github\Automacao_Web\venv\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


	Planilha com menores valores salva.

	O produto Memoria ram D41 8GB 3200 não entrou em promoção

	Loop realizado!
